In [1]:
import sys; sys.path.append('..'); sys.path.append('../..'); sys.path.append('../../..');
from data_models.fluxo_veiculos import get_fluxo_veiculos_df, get_fluxo_latest
from data_models.carteira import get_carteira_df, get_carteira_filters
df_fluxo = get_fluxo_latest()
df_carteira = get_carteira_df()

[CACHE] Hit for get_fluxo_latest (ad6dc8a746493181)
[CACHE] Hit for get_carteira_df (ad6dc8a746493181)


In [2]:
from api.services.allocators_service import *
def _get_filtered_cnpjs(
    df: pd.DataFrame, 
    client: Optional[str], 
    segment: Optional[str], 
    peer: Optional[str]
) -> set:
    """Helper to get CNPJs matching the filters."""
    if df.empty:
        return set()
    
    mask = pd.Series([True] * len(df), index=df.index)
    
    if client:
        mask &= df['cliente'] == client
    if segment and segment != 'all':
        mask &= df['cliente_segmentado'] == segment
    if peer and peer != 'all':
        mask &= df['peer'] == peer
    
    return set(df[mask]['cnpj_fundo'].unique())


In [ ]:
client = 'Itaú'
segment = None
peer = None
cnpjs_filter = _get_filtered_cnpjs(df_carteira, client, segment, peer)

# Filter carteira
df_cart_filtered = df_carteira[df_carteira['cnpj_fundo'].isin(cnpjs_filter)]

# -----------------------------------------------------
# 1. Flow by Window (same as performance)
# -----------------------------------------------------
flow_by_window = []
if not df_fluxo.empty:
    df_flow_filtered = df_fluxo[df_fluxo['cnpj_fundo'].isin(cnpjs_filter)]
    
    windows = [6, 12, 24, 36, 48, 60]
    for w in windows:
        col = f'fluxo_{w}m'
        if col in df_flow_filtered.columns:
            total = df_flow_filtered[col].sum()
            flow_by_window.append({
                'window': f'{w}M',
                'value': float(total) if pd.notnull(total) else 0
            })

# -----------------------------------------------------
# 2. Monthly Evolution by Gestor (stacked bar)
# -----------------------------------------------------
evolution_data = []
if not df_cart_filtered.empty:
    # Aggregate by (dt_comptc, gestor_cota)
    df_evo = df_cart_filtered.groupby(['dt_comptc', 'gestor_cota'])['vl_merc_pos_final'].sum().reset_index()
    
    # Pivot to have gestores as columns
    pivot = df_evo.pivot(index='dt_comptc', columns='gestor_cota', values='vl_merc_pos_final').fillna(0)
    
    # Get top N gestores by total value
    top_gestores = pivot.sum().nlargest(10).index.tolist()
    
    for dt in pivot.index:
        row = {'month': dt.strftime('%b/%y'), 'date': dt.isoformat()}
        for g in top_gestores:
            row[g] = float(pivot.loc[dt, g]) if g in pivot.columns else 0
        evolution_data.append(row)

# -----------------------------------------------------
# 3. Movement Difference (month over month change)
# -----------------------------------------------------
movement_diff = []
if not df_cart_filtered.empty:
    # Sum by month
    df_monthly = df_cart_filtered.groupby('dt_comptc')['vl_merc_pos_final'].sum().reset_index()
    df_monthly = df_monthly.sort_values('dt_comptc')
    df_monthly['diff'] = df_monthly['vl_merc_pos_final'].diff()
    
    for _, row in df_monthly.tail(12).iterrows():
        movement_diff.append({
            'month': row['dt_comptc'].strftime('%b/%y'),
            'value': float(row['diff']) if pd.notnull(row['diff']) else 0
        })

# -----------------------------------------------------
# 4. Current Snapshot (list of funds)
# -----------------------------------------------------
snapshot = []
pie_data = []

if not df_cart_filtered.empty:
    # Get latest date
    max_dt = df_cart_filtered['dt_comptc'].max()
    df_latest = df_cart_filtered[df_cart_filtered['dt_comptc'] == max_dt]
    
    # Aggregate by fund invested
    df_snap = df_latest.groupby(['cnpj_fundo_cota', 'nm_fundo_cota', 'peer']).agg({
        'vl_merc_pos_final': 'sum'

        2808
    }).reset_index()
    
    df_snap = df_snap.sort_values('vl_merc_pos_final', ascending=False).head(20)
    
    total_pl = df_snap['vl_merc_pos_final'].sum()
    
    for idx, row in df_snap.iterrows():
        pct = (row['vl_merc_pos_final'] / total_pl * 100) if total_pl > 0 else 0
        
        snapshot.append({
            'symbol': (row['peer'] or 'N/A')[:3].upper(),
            'name': (row['nm_fundo_cota'] or row['cnpj_fundo_cota'])[:40],
            'desc': row['peer'] or 'Fundo',
            'pl': float(row['vl_merc_pos_final']),
            'value': round(pct, 2)
        })
        
        pie_data.append({
            'name': (row['nm_fundo_cota'] or row['cnpj_fundo_cota'])[:30],
            'value': round(pct, 2),
            'pl': float(row['vl_merc_pos_final'])
        })

In [4]:
flow_by_window

[{'window': '6M', 'value': 5774480744.699997},
 {'window': '12M', 'value': -6132394154.359996},
 {'window': '24M', 'value': -44089507952.85999},
 {'window': '36M', 'value': -17165725344.810001},
 {'window': '48M', 'value': -35049630961.69},
 {'window': '60M', 'value': -29192816139.579994}]